In [22]:
%pylab inline
import pandas as pd
import matlab
import matlab.engine
MATLAB = matlab.engine.start_matlab()

Populating the interactive namespace from numpy and matplotlib


In [24]:
def selfN(x,B,C):
    return -B*x + C*tanh(x)

# Dynamics in general

We know that for any graph system, the node signals follow:

$$
 \frac{d \vec{x}}{dt} = M_0(\vec{x}) + diag(M_1(\vec{x}))A M_2(\vec{x}) 
$$

Using these dynamics, it should be theoretically possible to learn back the adjacency matrix of the network that produced the signals.   

In [25]:
Dynamics = { "R1": (lambda x: -1*x , lambda x: 1 , lambda x: x/(1+x)) , "R2":(lambda x: -(x)**(0.4) , 1 , lambda x: (x**(0.2)/(1 + x**(0.2)))), "P":(lambda x: - x**(0.5) , 1 , lambda x: x**(0.2)) , "M":(lambda x: x(1- x*x) , lambda x: x , lambda x: x/(1+x)) , "E":(lambda x: -1*x , lambda x: x**(0.5) , lambda x: x**2) , "N":(selfN , 1 , lambda x: tanh(x)) }

In [26]:
Type = "P"
M0 = Dynamics[Type][0]
M1 = Dynamics[Type][1]
M2 = Dynamics[Type][2]